In [1]:
# import sys
# !{sys.executable} -m pip install pytz
# !{sys.executable} -m pip install tzlocal
# !{sys.executable} -m pip install geopy

In [2]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime
import pytz
from tzlocal import get_localzone
from geopy import distance
from IPython.display import Markdown, Latex
from vending_token import load_vending_token

In [3]:
access_token = load_vending_token()
# access_token

In [4]:
def auth_headers(token = access_token):
    return { "Authorization": f"Bearer {token}" }

In [5]:
vending_url = "https://api.byu.edu/domains/vending/v1/vending.ashx"

In [6]:
category_list_params = {
    "format": "json",
    "service": "merchandise",
    "action": "listCategories"
}

category_list_response = requests.get(vending_url, headers = auth_headers(), params = category_list_params)

In [7]:
category_list = category_list_response.json()
category_list

[{'id': 1000, 'description': 'Food/Snacks'},
 {'id': 1001, 'description': 'Drinks'},
 {'id': 1002, 'description': 'Candy'},
 {'id': 1003, 'description': 'Ice Cream/Novelties'}]

In [8]:
target_category = 1000

In [9]:
product_list_params = {
    "format": "json",
    "service": "merchandise",
    "action": "getProducts",
    "cat": target_category
}

product_list_response = requests.get(vending_url, headers = auth_headers(), params = product_list_params)

In [10]:
product_list = product_list_response.json()
product_list[:5]

[{'id': 5243,
  'description': 'Sunbelt Choc. Chip Granola Bar',
  'price': '50',
  'img': ''},
 {'id': 5302, 'description': "Gardetto's", 'price': '80', 'img': ''},
 {'id': 5303,
  'description': 'Sun Chips Harvest Cheddar',
  'price': '100',
  'img': ''},
 {'id': 5304, 'description': 'Rold Gold Pretzels', 'price': '100', 'img': ''},
 {'id': 5306, 'description': 'Doritos Nacho Chips', 'price': '100', 'img': ''}]

In [11]:
sandwiches = [x for x in product_list if "ciabatta" in x["description"].lower()]
sandwiches

[{'id': 6114,
  'description': 'Italian Ciabatta Sandwich',
  'price': '350',
  'img': ''},
 {'id': 6151,
  'description': 'Chicken Bacon Ranch Ciabatta',
  'price': '350',
  'img': ''},
 {'id': 6916,
  'description': 'Chicken Poblano Toasted Ciabatta',
  'price': '350',
  'img': ''}]

In [12]:
byu_sandwich_ids = [
    6114, #Italian Ciabatta
    6151, #Chicken Bacon Ranch Ciabatta
    6196, #Chicken Poblano Toasted Ciabatta
]

In [13]:
croissants = [x for x in product_list if "croissant" in x["description"].lower()]
croissants

[{'id': 5985,
  'description': 'Chicken Salad Croissant',
  'price': '350',
  'img': ''},
 {'id': 6050, 'description': 'Ham Croissant', 'price': '350', 'img': ''},
 {'id': 6476,
  'description': 'Croissant Sausage Egg and Cheese',
  'price': '350',
  'img': ''},
 {'id': 6592,
  'description': 'Turkey Gouda Croissant',
  'price': '350',
  'img': ''}]

In [14]:
byu_croissant_ids = [
    5985, # Chicken Salad
    6592, # Turkey Gouda
    6050, # Ham & Swiss
]

In [15]:
bagels = [x for x in product_list if "bagel" in x["description"].lower()]
bagels

[{'id': 5977,
  'description': 'Blueberry BAGELw/ Cream Cheese',
  'price': '125',
  'img': ''},
 {'id': 5978,
  'description': 'Plain Bagel w/ Cream Cheese',
  'price': '125',
  'img': ''},
 {'id': 5996, 'description': 'Pizza Bagel', 'price': '125', 'img': ''},
 {'id': 5997, 'description': 'Asiago Bagel', 'price': '125', 'img': ''}]

In [16]:
byu_bagel_ids = [
    5997, # Asiago
    5977, # Blueberry
    5978, # Plain
    5996, # Pizza
]

Nutritional reference:
- sandwich(es??) https://dining.byu.edu/vending/images/nutrition/nutrition_Sandwiches.png
- croissants https://dining.byu.edu/vending/images/nutrition/nutrition_Croissants.png
- bagels (except pizza bagel?) https://dining.byu.edu/vending/images/nutrition/nutrition_Bagels.png

In [17]:
nutrition_data = pd.read_csv('./bakery_nutrition.csv')
nutrition_data[:3]

id  cost  calories  protein  carbohydrates  sodium  fiber  sugar   fat
0  6114   350       645       31             57    1903      2      5  34.0
1  5985   350       772       24             92    1061      6     36  46.0
2  6592   350       857       33             67    1862      2      9  60.0

Taken from [FDA daily recommended values document](https://www.fda.gov/media/99059/download) for ages >=4:

In [18]:
drv_data = pd.read_csv('./fda_drv.csv')
drv_data

name unit  quantity
0            fat    g        78
1  saturated_fat    g        20
2    cholesterol   mg       300
3  carbohydrates    g       275
4         sodium   mg      2300
5          fiber    g        28
6        protein    g        50
7          sugar    g        50

In [19]:
drv_data.loc[drv_data['name'] == 'sugar']

name unit  quantity
7  sugar    g        50

In [23]:
nutrition_data['name'] = nutrition_data.apply(lambda row: next(x['description'] for x in product_list if x["id"] == int(row.id)), axis = 1)
nutrition_data['calories_dollar'] = nutrition_data.apply(lambda row: row.calories / (row.cost / 100), axis = 1)
nutrition_data['protein_dollar'] = nutrition_data.apply(lambda row: row.protein / (row.cost / 100), axis = 1)
nutrition_data['fiber_dollar'] = nutrition_data.apply(lambda row: row.fiber / (row.cost / 100), axis = 1)
nutrition_data['protein_drv'] = nutrition_data.apply(lambda row: row.protein / drv_data.loc[drv_data['name'] == 'protein'].quantity, axis = 1)
nutrition_data['fiber_drv'] = nutrition_data.apply(lambda row: row.fiber / drv_data.loc[drv_data['name'] == 'fiber'].quantity, axis = 1)
nutrition_data[:3]

id  cost  calories  protein  carbohydrates  sodium  fiber  sugar   fat  \
0  6114   350       645       31             57    1903      2      5  34.0   
1  5985   350       772       24             92    1061      6     36  46.0   
2  6592   350       857       33             67    1862      2      9  60.0   

                        name  calories_dollar  protein_dollar  fiber_dollar  \
0  Italian Ciabatta Sandwich       184.285714        8.857143      0.571429   
1    Chicken Salad Croissant       220.571429        6.857143      1.714286   
2     Turkey Gouda Croissant       244.857143        9.428571      0.571429   

   protein_drv  fiber_drv  
0         0.62   0.071429  
1         0.48   0.214286  
2         0.66   0.071429

In [25]:
highest_calories_per_dollar = nutrition_data.sort_values(by=['calories_dollar'], ascending=False)
highest_calories_per_dollar[:5]

id  cost  calories  protein  carbohydrates  sodium  fiber  sugar   fat  \
5  5997   125       316       13             60     602      3      0   2.5   
6  5977   125       315       11             67     545      3     11   0.0   
2  6592   350       857       33             67    1862      2      9  60.0   
7  5978   125       289       12             60     531      3      0   0.0   
4  6146   350       790       17            133    1201     16     19  37.0   

                             name  calories_dollar  protein_dollar  \
5                    Asiago Bagel       252.800000       10.400000   
6  Blueberry BAGELw/ Cream Cheese       252.000000        8.800000   
2          Turkey Gouda Croissant       244.857143        9.428571   
7     Plain Bagel w/ Cream Cheese       231.200000        9.600000   
4    Wrap Turmeric Curry Garbanzo       225.714286        4.857143   

   fiber_dollar  protein_drv  fiber_drv  
5      2.400000         0.26   0.107143  
6      2.400000         0.22   0.107143  
2      0.571429         0.66   0.071429  
7      2.400000         0.24   0.107143  
4      4.571429         0.34   0.571429

In [26]:
highest_protein_per_dollar = nutrition_data.sort_values(by=['protein_dollar'], ascending=False)
highest_protein_per_dollar[:5]

id  cost  calories  protein  carbohydrates  sodium  fiber  sugar   fat  \
5  5997   125       316       13             60     602      3      0   2.5   
7  5978   125       289       12             60     531      3      0   0.0   
2  6592   350       857       33             67    1862      2      9  60.0   
0  6114   350       645       31             57    1903      2      5  34.0   
6  5977   125       315       11             67     545      3     11   0.0   

                             name  calories_dollar  protein_dollar  \
5                    Asiago Bagel       252.800000       10.400000   
7     Plain Bagel w/ Cream Cheese       231.200000        9.600000   
2          Turkey Gouda Croissant       244.857143        9.428571   
0       Italian Ciabatta Sandwich       184.285714        8.857143   
6  Blueberry BAGELw/ Cream Cheese       252.000000        8.800000   

   fiber_dollar  protein_drv  fiber_drv  
5      2.400000         0.26   0.107143  
7      2.400000         0.24   0.107143  
2      0.571429         0.66   0.071429  
0      0.571429         0.62   0.071429  
6      2.400000         0.22   0.107143

In [27]:
search_term = "gouda"

In [28]:
target_product = next(x for x in product_list if search_term.lower() in x["description"].lower())
target_product

{'id': 6592,
 'description': 'Turkey Gouda Croissant',
 'price': '350',
 'img': ''}

In [29]:
target_product_id = target_product['id']
target_product_id

6592

In [30]:
sandwiches

[{'id': 6114,
  'description': 'Italian Ciabatta Sandwich',
  'price': '350',
  'img': ''},
 {'id': 6151,
  'description': 'Chicken Bacon Ranch Ciabatta',
  'price': '350',
  'img': ''},
 {'id': 6916,
  'description': 'Chicken Poblano Toasted Ciabatta',
  'price': '350',
  'img': ''}]

This is where it will help to have your accurate location:

In [31]:
ex_lat_long = [40.249304, -111.651168]

In [32]:
in_stock_params = {
    "format": "json",
    "service": "inventory",
    "action": "findMachinesWithStock",
    "product": target_product_id
}

in_stock_response = requests.get(vending_url, headers = auth_headers(), params = in_stock_params)

In [33]:
in_stock_list = in_stock_response.json()
in_stock_list[:2]

[{'location': '5010',
  'description': '101 BRWB',
  'lat': '40.24673',
  'lng': '-111.64532',
  'items': [{'updateTime': '2020-10-0115:40:00.0-06:00',
    'price': '350',
    'amount': 3,
    'item': 'Turkey Gouda Croissant',
    'img': ''}]},
 {'location': '5011',
  'description': 'BRMB 144',
  'lat': '40.24632',
  'lng': '-111.65241',
  'items': [{'updateTime': '2020-10-0214:04:00.0-06:00',
    'price': '350',
    'amount': 2,
    'item': 'Turkey Gouda Croissant',
    'img': ''}]}]

In [34]:
def item_dist(lat_lng, item):
    item_lat_lng = list(map(float, [item["lat"], item["lng"]]))
    return distance.distance(np.array(item_lat_lng), np.array(lat_lng))

In [35]:
def add_distances_to_item(lat_lng, item):
    dist = item_dist(lat_lng, item)
    return {**item, "km": dist.km, "ft": dist.ft}

In [36]:
closest_items = list(map(lambda stock: add_distances_to_item(ex_lat_long, stock), in_stock_list))
# closest_items[:2]

In [37]:
closest_items_sorted = sorted(closest_items, key=lambda stock: stock["km"])
# closest_items_sorted[:2]

In [38]:
def pretty_vending_time(time_str):
    before_dot = time_str.split('.')[0]
    time_str = f"{before_dot[:10]}T{before_dot[10:]}-{time_str.split('-')[-1]}"
    local_tz = get_localzone()
    return datetime.fromisoformat(time_str).astimezone(local_tz).strftime("%x %X")

In [39]:
closest_stock_item = closest_items_sorted[0]
closest_stock_item

{'location': '5090',
 'description': 'TMCB 121',
 'lat': '40.24931',
 'lng': '-111.65058',
 'items': [{'updateTime': '2020-10-0208:38:00.0-06:00',
   'price': '350',
   'amount': 3,
   'item': 'Turkey Gouda Croissant',
   'img': ''}],
 'km': 0.05003294100493905,
 'ft': 164.150068913842}

In [40]:
closest_stock_item_details = closest_stock_item["items"][0]
closest_stock_item_details

{'updateTime': '2020-10-0208:38:00.0-06:00',
 'price': '350',
 'amount': 3,
 'item': 'Turkey Gouda Croissant',
 'img': ''}

In [41]:
closest_stock_item_update_time_str = pretty_vending_time(closest_stock_item_details["updateTime"])
closest_stock_item_update_time_str

'10/02/20 08:38:00'

In [42]:
closest_nutrition_info = nutrition_data.loc[nutrition_data['id'] == target_product_id]
if not closest_nutrition_info.empty:
    closest_calories = int(closest_nutrition_info['calories'])
    closest_protein = int(closest_nutrition_info['protein'])
    closest_fiber = int(closest_nutrition_info['fiber'])
    closest_protein_drv = '{0:.02f}'.format(round(float(closest_nutrition_info['protein_drv']) * 100, 2))
    closest_fiber_drv = '{0:.02f}'.format(round(float(closest_nutrition_info['fiber_drv']) * 100, 2))
    closest_calories_per_dollar = round(float(closest_nutrition_info['calories_dollar']), 2)
    closest_protein_per_dollar = round(float(closest_nutrition_info['protein_dollar']), 2)
    closest_fiber_per_dollar = round(float(closest_nutrition_info['fiber_dollar']), 2)

In [43]:
closest_name = closest_stock_item_details['item']
closest_ft_dist = round(closest_stock_item['ft'])
closest_location_name = closest_stock_item['description']
closest_quantity = closest_stock_item_details['amount']
closest_price = '{0:.02f}'.format(int(closest_stock_item_details['price']) / 100)

nutrition_str = ""

if not closest_nutrition_info.empty:
    nutrition_str = f"""
Contains:
- {closest_calories} calories
- {closest_protein}g protein ({closest_protein_drv}% daily recommended value)
- {closest_fiber}g fiber ({closest_fiber_drv}% daily recommended value)
- {closest_calories_per_dollar} calories/dollar
- {closest_protein_per_dollar}g protein/dollar
- {closest_fiber_per_dollar}g fiber/dollar
"""

info_str = f"""
### The closest {closest_name} at BYU is {closest_ft_dist} ft away, at {closest_location_name}, for ${closest_price}.

{nutrition_str}

At __{closest_stock_item_update_time_str}__, there {"were" if closest_quantity != 1 else "was"}
__{closest_quantity}__ available.
"""

display(Markdown(info_str))


### The closest Turkey Gouda Croissant at BYU is 164 ft away, at TMCB 121, for $3.50.


Contains:
- 857 calories
- 33g protein (66.00% daily recommended value)
- 2g fiber (7.14% daily recommended value)
- 244.86 calories/dollar
- 9.43g protein/dollar
- 0.57g fiber/dollar


At __10/02/20 08:38:00__, there were
__3__ available.
